# ZeroScript: AI-Powered Browser Test Automation using PraisonAI Agent & OpenAI

# Description

AI-powered browser testing using PraisonAI Agent and OpenAI. Executes YAML-defined test cases, captures screenshots, logs interactions, and generates JSON reports. Ideal for intelligent, automated UI testing.



[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DhivyaBharathy-web/PraisonAI/blob/main/examples/cookbooks/ZeroScript_AI_TestExecutor.ipynb)


# Dependencies

In [34]:
!pip install --quiet praisonaiagents browser-use python-dotenv pyyaml openai


# Environment Setup

In [35]:
import os
from dotenv import load_dotenv

os.environ["OPENAI_API_KEY"] = "Enter your key"  # replace with actual key
load_dotenv()


True

# Create my_secrets.py

In [36]:
with open("my_secrets.py", "w") as f:
    f.write('''sensitive_data = {
    "username": "standard_user",
    "password": "secret_sauce"
}''')


# Create views.py

In [37]:
with open("views.py", "w") as f:
    f.write('''
from pydantic import BaseModel

class TestResult(BaseModel):
    status: str
    message: str = ""
''')


# Create agent.py

In [38]:
with open("agent.py", "w") as f:
    f.write('''
import os
from praisonaiagents import Agent
from browser_use import Browser, BrowserConfig, Controller
from browser_use.browser.context import BrowserContext, BrowserContextConfig
from openai import OpenAI
from views import TestResult
from my_secrets import sensitive_data

# Browser setup
browser = Browser(config=BrowserConfig(headless=True))
context_cfg = BrowserContextConfig(save_recording_path="data/recordings")
controller = Controller(output_model=TestResult)

async def browser_use(task: str):
    context = BrowserContext(browser=browser, config=context_cfg)
    agent = Agent(
        task=task,
        llm=OpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-4"),
        browser_context=context,
        sensitive_data=sensitive_data,
        controller=controller,
        save_conversation_path="data/conversations/convo",
        extend_system_message=(
            "You are a professional software tester. Execute browser test steps precisely."
        )
    )
    response = await agent.run()
    await context.close()
    return response
''')


# Create Sample Test YAML

In [39]:
from pathlib import Path
tests_dir = Path("tests")
tests_dir.mkdir(exist_ok=True)
yaml_content = """
hooks:
  beforeEach: |
    Navigate to https://www.saucedemo.com
  afterEach: |
    Take screenshot

tests:
  - id: "login_test_001"
    name: "Valid Login Test"
    instructions: |
      1. Enter standard_user in username field
      2. Enter secret_sauce in password field
      3. Click login button
      4. Verify page contains text "Products"
"""
(Path("tests/login_test.yml")).write_text(yaml_content)


350

# Run Tests

In [40]:
import asyncio, yaml, json, base64, shutil
from agent import browser_use, browser
from pathlib import Path
from datetime import datetime

# Clean or create data dirs
for sub in ["recordings", "conversations", "screenshots", "reports"]:
    d = Path("data") / sub
    if d.exists(): shutil.rmtree(d)
    d.mkdir(parents=True, exist_ok=True)

async def execute_test_file(path):
    cfg = yaml.safe_load(Path(path).read_text())
    hooks, results = cfg.get("hooks", {}), []
    for t in cfg["tests"]:
        task = f"{hooks.get('beforeEach','')}\n{t['instructions']}\n{hooks.get('afterEach','')}"
        res = {"id": t["id"], "name": t["name"]}
        try:
            r = await browser_use(task)
            imgs = r.screenshots()
            if imgs:
                out = Path("data/screenshots") / f"{t['id']}.png"
                out.write_bytes(base64.b64decode(imgs[-1]))
            result_obj = r.final_result() if hasattr(r, "final_result") else str(r)
            res["result"] = result_obj.dict() if hasattr(result_obj, "dict") else str(result_obj)
        except Exception as e:
            res["error"] = str(e)
        results.append(res)
    return results

async def main():
    all_results = []
    for file in Path("tests").glob("*.yml"):
        all_results.extend(await execute_test_file(str(file)))
    report = Path("data/reports") / f"report_{datetime.now():%Y%m%d_%H%M%S}.json"
    report.write_text(json.dumps(all_results, indent=2))
    await browser.close()
    print(f"✅ Report saved to {report}")

await main()


Exception ignored in: <function BrowserSession.__del__ at 0x7e2f540fb100>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/browser_use/browser/session.py", line 497, in __del__
    status = f'🪓 killing pid={self.browser_pid}...' if (self.browser_pid and owns_browser) else '☠️'
                                                         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pydantic/main.py", line 991, in __getattr__
    raise AttributeError(f'{type(self).__name__!r} object has no attribute {item!r}')
AttributeError: 'BrowserSession' object has no attribute 'browser_pid'


✅ Report saved to data/reports/report_20250630_092319.json
